### this is just to check functionalities

#### lets run s&p 500 code in parallel in fragments


In [41]:
import pandas as pd
import torch


import yfinance as yf
import pandas_ta as ta
import os


In [66]:
# Define a function to format numbers based on their value
def custom_format(value):
    if abs(value) >= 10000:
        # Format numbers >= 100,000 with commas and no decimals
        return f'{value:,.0f}'
    elif abs(value) >= 1000:
        # Format numbers > 1000 without commas and decimals
        return f'{value:.0f}'
    else:
        return f'{value:.2f}'

# Define a styling function for red negative numbers with parentheses
def format_negative_red(value):
    if value < 0:
        return f'color: red; font-weight: bold;'  # CSS for red color
    return None

def format_negative_parentheses(value):
    formatted_value = custom_format(value)
    return f'({formatted_value})' if value < 0 else formatted_value

def display_df(df):
    return df.style.applymap(format_negative_red).format(format_negative_parentheses)

In [67]:
# Function to get S&P 500 constituents
def get_sp500_constituents():
    sp500_url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    df = pd.read_html(sp500_url, header=0)[0]
    return df


# Function to calculate weights based on market cap
def calculate_weights(df):
    df['Weight'] = df['Market Cap'] / df['Market Cap'].sum() * 100
    return df


# function to deal with multi-level index
def organize_columns(stock_data):
    refined_stock_data = stock_data.reset_index()
    # test_df = analysis_results['AAPL']['Close'].reset_index()
    refined_stock_data.columns = ['Date', 'Close']
    refined_stock_data.set_index(['Date'], inplace=True)
    
    return refined_stock_data


# Function to get stock data and technical analysis
def analyze_stock(stock_symbol, start_date, end_date):
    # Download historical data from Yahoo Finance
    stock_data = yf.download(stock_symbol, start=start_date, end=end_date)
    close_px = organize_columns(stock_data=stock_data['Close'])
    # display(close_px)

    # Calculate technical indicators
    stock_data['SMA_50'] = ta.sma(close_px['Close'], length=50)  # 50-day Simple Moving Average
    # print(ta.sma(close_px['Close'], length=50))
    stock_data['SMA_200'] = ta.sma(close_px['Close'], length=200)  # 200-day Simple Moving Average
    stock_data['RSI'] = ta.rsi(close_px['Close'], length=14)  # Relative Strength Index

    # MACD
    macd = ta.macd(close_px['Close'])
    stock_data['MACD'] = macd['MACD_12_26_9']
    stock_data['Signal'] = macd['MACDs_12_26_9']
    stock_data['Histogram'] = macd['MACDh_12_26_9']

    return stock_data


In [71]:
def get_sp500():
    sp500_df = get_sp500_constituents()

    sp500_df['Market Cap'] = [100000 + i * 1000 for i in range(len(sp500_df))] # just random numbers
    sp500_df = calculate_weights(sp500_df)

    # folder_name = "data"
    # if not os.path.exists(folder_name):
    #     os.makedirs(folder_name)

    return sp500_df

# Analyze each stock
def analyse_stock(symbol):
    start_date = "2023-01-01"
    end_date = "2023-12-01"
    analysis_results = {}

    # for symbol in sp500_df['Symbol']:
    try:
        stock_symbol = f"{symbol}"  # S&P 500 symbols don't require suffixes like ".NS"
        stock_analysis = analyze_stock(stock_symbol, start_date, end_date)
        analysis_results[symbol] = stock_analysis
        print(f"Analyzed {symbol}")
        return stock_analysis
    
    except Exception as e:
        print(f"Error analyzing {symbol}: {e}")

    


In [78]:
# sp500_df['Symbol'].sample(5)
# stock_symbol = "AAPL"
output_folder = os.path.join("data", "us_markets")
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Save the SP500 constituents to a CSV file in the created subfolder
output_file = os.path.join(output_folder, f"sp500_constituents.csv")
# override = False
if os.path.exists(output_file):
    sp500_df = pd.read_csv(output_file)
    print(f"SP500 loaded from {output_file}")
else:
    sp500_df = get_sp500()
    sp500_df.to_csv(output_file)
    print(f"SP500 saved to {output_file}")

analysis_output_folder = os.path.join("data", "us_markets", "analysis")
if not os.path.exists(analysis_output_folder):
    os.makedirs(analysis_output_folder)

analysis_results = {}
for symbol in sp500_df['Symbol']:
    output_file = os.path.join(analysis_output_folder, f"{symbol}_analysis.csv")
    if os.path.exists(output_file):
        temp_df = pd.read_csv(output_file)
        analysis_results[symbol] = temp_df
        print(f"Analysis of {symbol} loaded from {output_file}")
    else:
        temp_df = analyse_stock(symbol=symbol)
        if temp_df is not None:
            temp_df.to_csv(output_file)
            analysis_results[symbol] = temp_df
            print(f"Analysis of {symbol} saved to {output_file}")
        else:
            print(f"Analysis of {symbol} failed")





SP500 loaded from data/us_markets/sp500_constituents.csv
Analysis of MMM loaded from data/us_markets/analysis/MMM_analysis.csv
Analysis of AOS loaded from data/us_markets/analysis/AOS_analysis.csv
Analysis of ABT loaded from data/us_markets/analysis/ABT_analysis.csv
Analysis of ABBV loaded from data/us_markets/analysis/ABBV_analysis.csv
Analysis of ACN loaded from data/us_markets/analysis/ACN_analysis.csv
Analysis of ADBE loaded from data/us_markets/analysis/ADBE_analysis.csv
Analysis of AMD loaded from data/us_markets/analysis/AMD_analysis.csv
Analysis of AES loaded from data/us_markets/analysis/AES_analysis.csv
Analysis of AFL loaded from data/us_markets/analysis/AFL_analysis.csv
Analysis of A loaded from data/us_markets/analysis/A_analysis.csv
Analysis of APD loaded from data/us_markets/analysis/APD_analysis.csv
Analysis of ABNB loaded from data/us_markets/analysis/ABNB_analysis.csv
Analysis of AKAM loaded from data/us_markets/analysis/AKAM_analysis.csv
Analysis of ALB loaded from d

[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['AMTM']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2023-01-01 -> 2023-12-01) (Yahoo error = "Data doesn\'t exist for startDate = 1672549200, endDate = 1701406800")')


Error analyzing AMTM: 'NoneType' object is not subscriptable
Analysis of AMTM failed
Analysis of AEE loaded from data/us_markets/analysis/AEE_analysis.csv
Analysis of AEP loaded from data/us_markets/analysis/AEP_analysis.csv
Analysis of AXP loaded from data/us_markets/analysis/AXP_analysis.csv
Analysis of AIG loaded from data/us_markets/analysis/AIG_analysis.csv
Analysis of AMT loaded from data/us_markets/analysis/AMT_analysis.csv
Analysis of AWK loaded from data/us_markets/analysis/AWK_analysis.csv
Analysis of AMP loaded from data/us_markets/analysis/AMP_analysis.csv
Analysis of AME loaded from data/us_markets/analysis/AME_analysis.csv
Analysis of AMGN loaded from data/us_markets/analysis/AMGN_analysis.csv
Analysis of APH loaded from data/us_markets/analysis/APH_analysis.csv
Analysis of ADI loaded from data/us_markets/analysis/ADI_analysis.csv
Analysis of ANSS loaded from data/us_markets/analysis/ANSS_analysis.csv
Analysis of AON loaded from data/us_markets/analysis/AON_analysis.csv
A

[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BRK.B']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')
[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['BF.B']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2023-01-01 -> 2023-12-01)')


Error analyzing BRK.B: 'NoneType' object is not subscriptable
Analysis of BRK.B failed
Analysis of BBY loaded from data/us_markets/analysis/BBY_analysis.csv
Analysis of TECH loaded from data/us_markets/analysis/TECH_analysis.csv
Analysis of BIIB loaded from data/us_markets/analysis/BIIB_analysis.csv
Analysis of BLK loaded from data/us_markets/analysis/BLK_analysis.csv
Analysis of BX loaded from data/us_markets/analysis/BX_analysis.csv
Analysis of BK loaded from data/us_markets/analysis/BK_analysis.csv
Analysis of BA loaded from data/us_markets/analysis/BA_analysis.csv
Analysis of BKNG loaded from data/us_markets/analysis/BKNG_analysis.csv
Analysis of BWA loaded from data/us_markets/analysis/BWA_analysis.csv
Analysis of BSX loaded from data/us_markets/analysis/BSX_analysis.csv
Analysis of BMY loaded from data/us_markets/analysis/BMY_analysis.csv
Analysis of AVGO loaded from data/us_markets/analysis/AVGO_analysis.csv
Analysis of BR loaded from data/us_markets/analysis/BR_analysis.csv
Ana

...

Error analyzing GEV: 'NoneType' object is not subscriptable
Analysis of GEV failed
Analysis of GEN loaded from data/us_markets/analysis/GEN_analysis.csv
Analysis of GNRC loaded from data/us_markets/analysis/GNRC_analysis.csv
Analysis of GD loaded from data/us_markets/analysis/GD_analysis.csv
Analysis of GIS loaded from data/us_markets/analysis/GIS_analysis.csv
Analysis of GM loaded from data/us_markets/analysis/GM_analysis.csv
Analysis of GPC loaded from data/us_markets/analysis/GPC_analysis.csv
Analysis of GILD loaded from data/us_markets/analysis/GILD_analysis.csv
Analysis of GPN loaded from data/us_markets/analysis/GPN_analysis.csv
Analysis of GL loaded from data/us_markets/analysis/GL_analysis.csv
Analysis of GDDY loaded from data/us_markets/analysis/GDDY_analysis.csv
Analysis of GS loaded from data/us_markets/analysis/GS_analysis.csv
Analysis of HAL loaded from data/us_markets/analysis/HAL_analysis.csv
Analysis of HIG loaded from data/us_markets/analysis/HIG_analysis.csv
Analysis 

[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['SW']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2023-01-01 -> 2023-12-01) (Yahoo error = "Data doesn\'t exist for startDate = 1672549200, endDate = 1701406800")')


Error analyzing SW: 'NoneType' object is not subscriptable
Analysis of SW failed
Analysis of SNA loaded from data/us_markets/analysis/SNA_analysis.csv


[*********************100%***********************]  1 of 1 completed
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['SOLV']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2023-01-01 -> 2023-12-01) (Yahoo error = "Data doesn\'t exist for startDate = 1672549200, endDate = 1701406800")')


Error analyzing SOLV: 'NoneType' object is not subscriptable
Analysis of SOLV failed
Analysis of SO loaded from data/us_markets/analysis/SO_analysis.csv
Analysis of LUV loaded from data/us_markets/analysis/LUV_analysis.csv
Analysis of SWK loaded from data/us_markets/analysis/SWK_analysis.csv
Analysis of SBUX loaded from data/us_markets/analysis/SBUX_analysis.csv
Analysis of STT loaded from data/us_markets/analysis/STT_analysis.csv
Analysis of STLD loaded from data/us_markets/analysis/STLD_analysis.csv
Analysis of STE loaded from data/us_markets/analysis/STE_analysis.csv
Analysis of SYK loaded from data/us_markets/analysis/SYK_analysis.csv
Analysis of SMCI loaded from data/us_markets/analysis/SMCI_analysis.csv
Analysis of SYF loaded from data/us_markets/analysis/SYF_analysis.csv
Analysis of SNPS loaded from data/us_markets/analysis/SNPS_analysis.csv
Analysis of SYY loaded from data/us_markets/analysis/SYY_analysis.csv
Analysis of TMUS loaded from data/us_markets/analysis/TMUS_analysis.c

In [ ]:
# # Save results
# sp500_df.to_csv("sp500_constituents.csv", index=False)
# for symbol, analysis in analysis_results.items():
#     analysis.to_csv(f"{symbol}_analysis.csv")

# print("Analysis complete. Files saved.")

# Save results
# file_path = os.path.join(folder_name, "sp500_constituents.csv")
# sp500_df.to_csv(file_path, index=False)
# for symbol, analysis in analysis_results.items():
#     temp_path = os.path.join(folder_name, f"{symbol}_analysis.csv")
#     analysis.to_csv(temp_path)

# print("Analysis complete. Files saved.")



In [ ]:
# # need to write code to load data and results

# # stock_symbol = "AAPL"
# output_folder = os.path.join("data", "us_markets")
# if not os.path.exists(output_folder):
#     os.makedirs(output_folder)

# # Save the SP500 constituents to a CSV file in the created subfolder
# output_file = os.path.join(output_folder, f"sp500_constituents.csv")
# override = False
# if os.path.exists(output_file) and not override:
#     gdp_df = pd.read_csv(output_file)
#     print(f"SP500 loaded from {output_file}")
# else:
#     gdp_df = get_aapl_data()
#     gdp_df.to_csv(output_file)
#     print(f"SP500 saved to {output_file}")


In [5]:
import yfinance as yf
import plotly.graph_objs as go

def create_market_cap_chart(symbol, start_date, end_date):
    # Fetch historical data
    stock = yf.Ticker(symbol)
    hist = stock.history(start=start_date, end=end_date)
    
    # Calculate market cap (assuming shares outstanding remains constant)
    shares_outstanding = stock.info['sharesOutstanding']
    hist['Market Cap'] = hist['Close'] * shares_outstanding
    
    # Create the chart
    fig = go.Figure()
    
    # Add price line
    fig.add_trace(go.Scatter(
        x=hist.index,
        y=hist['Close'],
        name='Stock Price',
        line=dict(color='blue')
    ))
    
    # Add market cap line
    fig.add_trace(go.Scatter(
        x=hist.index,
        y=hist['Market Cap'],
        name='Market Cap',
        yaxis='y2',
        line=dict(color='green')
    ))
    
    # Update layout
    fig.update_layout(
        title=f'{symbol} Stock Price and Market Cap Over Time',
        xaxis_title='Date',
        yaxis_title='Stock Price (USD)',
        yaxis2=dict(
            title='Market Cap (USD)',
            overlaying='y',
            side='right'
        ),
        legend=dict(x=0, y=1, traceorder='normal')
    )
    
    return fig

# Example usage
symbol = 'AAPL'
start_date = '2020-01-01'
end_date = '2024-12-22'
fig = create_market_cap_chart(symbol, start_date, end_date)
fig.show()


In [63]:
import yfinance as yf
import pandas as pd


def get_aapl_data():

    # Define the stock symbol (e.g., Apple: AAPL) and the time range
    stock_symbol = "AAPL"  # Replace with the stock symbol of your choice
    start_date = "1993-01-01"  # 30 years back from 2023
    end_date = "2023-12-31"

    # Fetch historical stock data
    stock_data = yf.Ticker(stock_symbol)
    historical_data = stock_data.history(start=start_date, end=end_date)

    # Fetch additional information like shares outstanding
    shares_outstanding = stock_data.info.get("sharesOutstanding", "N/A")

    # Format the historical data to include shares outstanding
    historical_data["Shares Outstanding"] = shares_outstanding

    # Save or display the data
    # display_df(historical_data.sample(50))

    return historical_data

stock_symbol = "AAPL"
output_folder = os.path.join("data", "us_markets")
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Save the GDP data to a CSV file in the created subfolder
output_file = os.path.join(output_folder, f"{stock_symbol}_historical_stock_data.csv")
if os.path.exists(output_file):
    gdp_df = pd.read_csv(output_file)
    print(f"GDP data loaded from {output_file}")
else:
    gdp_df = get_aapl_data()
    gdp_df.to_csv(output_file)
    print(f"GDP data saved to {output_file}")

# # Save to CSV (optional)
# historical_data.to_csv(f"{stock_symbol}_historical_stock_data.csv")


GDP data saved to data/us_markets/AAPL_historical_stock_data.csv


In [8]:
file_path = os.path.join(output_folder, "plot_aapl.html")

fig.write_html(file_path)
# folder_name


In [58]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime

# Define a function to fetch current S&P 500 constituents
def fetch_sp500_constituents():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    
    # Parse the table to get the list of companies
    df = pd.read_html(str(table))[0]
    return df['Symbol'].tolist()

# Fetch current constituents as a starting point
current_constituents = fetch_sp500_constituents()

# Placeholder for historical data collection logic
# This would typically involve accessing a financial data API or database
def fetch_historical_rebalance_data():
    # Example structure for storing historical data
    # In practice, this would be filled with real historical data
    historical_data = {
        '1999-03-31': ['AAPL', 'MSFT', 'GOOGL'],  # Example constituents at a past date
        '2000-06-30': ['AAPL', 'MSFT', 'AMZN'],   # Changes over time
        # ... more historical dates and constituents ...
    }
    return historical_data

# Analyze changes in constituents over time
def analyze_rebalance_changes(historical_data):
    changes = {}
    previous_constituents = set()
    
    for date, constituents in sorted(historical_data.items()):
        current_set = set(constituents)
        added = current_set - previous_constituents
        removed = previous_constituents - current_set
        
        changes[date] = {
            'added': added,
            'removed': removed
        }
        
        previous_constituents = current_set
    
    return changes

# Fetch historical rebalance data (this is a placeholder function)
historical_data = fetch_historical_rebalance_data()

# Analyze changes in the S&P 500 constituents over time
rebalance_changes = analyze_rebalance_changes(historical_data)

# Output the rebalance changes for inspection
for date, change in rebalance_changes.items():
    print(f"Date: {date}")
    print(f"Added: {change['added']}")
    print(f"Removed: {change['removed']}")
    print("\n")


Date: 1999-03-31
Added: {'GOOGL', 'AAPL', 'MSFT'}
Removed: set()


Date: 2000-06-30
Added: {'AMZN'}
Removed: {'GOOGL'}




/var/folders/_2/m6nqqq692z5gjqwq_qk2lcjr0000gn/T/ipykernel_1962/3118521692.py:14: FutureWarning:

Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.



#### this is with options data

In [ ]:
# Analyze each stock and options -- nothing came out of this
start_date = "2023-01-01"
end_date = "2023-12-01"
analysis_results = {}
options_results = {}

for symbol in sp500_df['Symbol'].sample(5):
    try:
        stock_symbol = f"{symbol}"  # S&P 500 symbols don't require suffixes like ".NS"

        # Analyze stock data
        stock_analysis = analyze_stock(stock_symbol, start_date, end_date)
        analysis_results[symbol] = stock_analysis

        print(f"Analyzed {symbol}")
    except Exception as e:
        print(f"Error analyzing {symbol}: {e}")

In [ ]:
# Main script
if __name__ == "__main__":
    try:
        # Get S&P 500 constituents
        sp500_df = get_sp500_constituents()
        
        # Add market capitalization column manually (replace this with API if available)
        sp500_df['Market Cap'] = [100000 + i * 1000 for i in range(len(sp500_df))]  # Placeholder values

        # Calculate index weights
        sp500_df = calculate_weights(sp500_df)

        # Analyze each stock
        start_date = "2023-01-01"
        end_date = "2023-12-01"
        analysis_results = {}
        options_results = {}

        for symbol in sp500_df['Symbol']:
            try:
                stock_symbol = f"{symbol}"  # S&P 500 symbols don't require suffixes like ".NS"

                # Analyze stock data
                stock_analysis = analyze_stock(stock_symbol, start_date, end_date)
                analysis_results[symbol] = stock_analysis

                # # Fetch options data
                # options_data = get_options_data(stock_symbol)
                # options_results[symbol] = options_data

                print(f"Analyzed {symbol}")
            except Exception as e:
                print(f"Error analyzing {symbol}: {e}")

        # Save results
        sp500_df.to_csv("sp500_constituents.csv", index=False)
        for symbol, analysis in analysis_results.items():
            analysis.to_csv(f"{symbol}_analysis.csv")

        # for symbol, options in options_results.items():
        #     with open(f"{symbol}_options.json", "w") as f:
        #         f.write(pd.DataFrame(options).to_json())

        print("Analysis complete. Files saved.")
    except Exception as e:
        print(f"Error: {e}")

### Creating proper visualization of WB gdp data

In [ ]:
# combining all parts here

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

import wbdata
import pandas as pd
import datetime


In [32]:
#  load actual gdp data 

def get_gdp_data():
    # Define the date range (past 35 years)
    start_date = datetime.datetime(1989, 1, 1)
    end_date = datetime.datetime(2024, 1, 1)

    # Define the top 25 economies by ISO country codes (based on search results)
    countries = [
        'US', 'CN', 'JP', 'DE', 'IN', 'GB', 'FR', 'BR', 'IT', 'CA',
        'KR', 'RU', 'AU', 'ES', 'MX', 'ID', 'NL', 'SA', 'TR', 'CH',
        'TW', 'PL', 'SE', 'TH', 'BE'
    ]

    # Define the GDP indicator (current US$)
    indicator = {'NY.GDP.MKTP.CD': 'GDP (current US$)'}

    # Retrieve GDP data from the World Bank API
    df = wbdata.get_dataframe(indicator, country=countries, date=(start_date, end_date), 
                            #   convert_date=False
                            )

    # Reshape the data for better readability
    df = df.reset_index()
    df_pivot = df.pivot(index='date', columns='country', values='GDP (current US$)')

    return df_pivot



output_folder = os.path.join("data", "gdp_data")
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Save the GDP data to a CSV file in the created subfolder
output_file = os.path.join(output_folder, "top_25_economies_gdp_1989_2024.csv")
if os.path.exists(output_file):
    gdp_df = pd.read_csv(output_file)
    print(f"GDP data loaded from {output_file}")
else:
    gdp_df = get_gdp_data()
    gdp_df.to_csv(output_file)
    print(f"GDP data saved to {output_file}")





GDP data loaded from data/gdp_data/top_25_economies_gdp_1989_2024.csv


In [ ]:
# df = pd.DataFrame(data)

# Melt the DataFrame for Plotly compatibility
df_melted = gdp_df.melt(id_vars=["date"], var_name="Country", value_name="GDP")

# # Sort the data dynamically by GDP within each year
# def reorder_categories(frame):
#     sorted_countries = frame.sort_values(by="GDP", ascending=False)["Country"].tolist()
#     frame["Country"] = pd.Categorical(frame["Country"], categories=sorted_countries, ordered=True)
#     return frame

# df_melted = df_melted.groupby("Year", group_keys=False).apply(reorder_categories)

# Create the Plotly animation
fig = px.bar(
    df_melted,
    x="Country",
    y="GDP",
    color="Country",
    animation_frame="date",
    title="Top 5 Economies Over Time",
    labels={"GDP": "GDP (Trillions USD)"},
    range_y=[0, gdp_df.drop('date', axis=1).max().max() * 1.1],
)

# Update layout for dynamic sorting of x-axis
fig.update_layout(
    xaxis_title="Country",
    yaxis_title="GDP (Trillions USD)",
    legend_title="Country",
    template="plotly_white",
    xaxis=dict(categoryorder="total descending"), # used for dynamically rearranging x axis
    xaxis_tickangle=45,  # Tilt the x-axis labels
)

# Show the interactive animation
fig.show()

In [54]:
display_df(gdp_df)
# gdp_df

/var/folders/_2/m6nqqq692z5gjqwq_qk2lcjr0000gn/T/ipykernel_1962/2330706337.py:23: FutureWarning:

Styler.applymap has been deprecated. Use Styler.map instead.



,date,Australia,Belgium,Brazil,Canada,China,France,Germany,India,Indonesia,Italy,Japan,"Korea, Rep.",Mexico,Netherlands,Poland,Russian Federation,Saudi Arabia,Spain,Sweden,Switzerland,Thailand,Turkiye,United Kingdom,United States
0,1989,"299,875,906,679","164,221,056,511","412,990,820,287","567,211,993,243","347,768,051,312","1,016,742,237,302","1,404,092,925,205","296,042,052,945","94,451,427,877","930,801,709,004","3,109,455,047,824","246,928,837,311","221,403,098,267","258,716,904,292",nan,"506,631,299,735","95,344,459,279","414,460,794,097","217,634,838,449","208,105,846,934","72,250,748,100","107,127,191,329","926,884,816,754","5,641,580,000,000"
1,1990,"311,426,665,220","205,331,747,948","464,989,098,145","596,075,591,361","360,857,912,566","1,257,649,439,827","1,778,162,195,860","320,979,026,420","106,140,727,334","1,183,945,130,899","3,185,904,656,664","283,365,844,161","261,253,675,693","318,799,003,994","65,977,748,211","517,014,446,228","117,630,173,565","536,175,326,477","261,469,578,631","265,763,573,999","85,343,190,719","150,655,500,192","1,093,169,389,205","5,963,144,000,000"
2,1991,"325,975,319,456","210,510,999,409","342,534,090,909","612,527,712,316","383,373,318,084","1,258,961,748,634","1,875,792,575,133","270,105,341,879","116,621,996,217","1,249,092,439,519","3,648,065,760,649","330,647,042,837","313,139,656,146","327,982,316,124","85,500,935,935","517,962,962,963","132,223,230,975","576,753,902,322","273,834,608,020","268,901,693,863","98,234,714,971","151,034,731,544","1,142,797,178,131","6,158,129,000,000"
3,1992,"325,525,379,567","234,781,652,447","328,191,909,882","594,387,358,319","426,915,712,716","1,389,663,073,110","2,141,377,582,968","288,208,070,278","128,026,966,580","1,323,204,350,354","3,980,702,922,118","355,524,903,068","363,157,832,924","363,497,050,125","94,337,050,693","460,290,556,901","137,087,850,467","630,465,353,486","283,912,173,667","279,921,425,828","111,452,746,518","159,104,772,992","1,179,659,529,660","6,520,327,000,000"
4,1993,"312,133,923,553","224,721,795,709","368,292,034,381","579,053,561,739","444,731,282,436","1,314,383,368,080","2,078,954,217,438","279,295,648,983","158,006,700,302","1,067,412,587,671","4,536,940,479,038","392,665,710,525","530,160,763,984","354,070,495,966","96,043,157,273","435,083,713,851","132,967,957,276","524,700,572,624","212,647,043,708","272,237,527,071","128,889,262,951","180,415,757,852","1,061,388,722,256","6,858,559,000,000"
5,1994,"322,806,641,301","244,884,129,491","525,369,467,296","579,944,346,807","564,321,854,521","1,385,822,778,828","2,215,282,632,277","327,274,843,459","176,892,143,932","1,101,750,159,702","4,998,797,547,741","463,619,823,515","553,618,247,901","379,688,232,232","110,803,635,288","395,077,301,248","135,174,899,866","530,183,651,720","228,701,658,891","301,375,055,202","146,683,778,959","130,650,447,499","1,140,489,745,944","7,287,236,000,000"
6,1995,"368,166,023,166","288,025,588,396","769,333,050,987","605,961,090,061","734,484,834,574","1,595,219,345,512","2,593,053,091,306","360,281,909,643","202,132,028,723","1,177,369,428,266","5,545,563,663,890","566,581,003,128","380,157,469,867","452,967,334,614","142,838,527,115","395,537,185,735","143,343,124,166","614,170,002,669","267,052,836,695","352,835,806,342","169,278,916,593","169,319,579,016","1,344,240,176,740","7,639,749,000,000"
7,1996,"401,341,880,621","279,201,433,225","850,426,432,992","630,607,994,133","863,749,314,719","1,598,889,216,566","2,506,576,553,158","392,896,866,205","227,369,679,375","1,314,776,508,972","4,923,391,533,852","610,167,053,824","432,157,945,024","451,372,549,020","160,813,026,223","391,724,890,744","158,662,483,311","642,251,412,058","291,952,430,659","340,103,959,547","183,035,237,429","181,464,408,820","1,419,645,865,835","8,073,122,000,000"
8,1997,"435,642,611,297","252,708,051,421","883,206,179,730","654,986,999,856","961,601,980,985","1,449,392,222,971","2,218,790,886,533","415,867,563,593","215,748,998,610","1,244,988,176,4

In [44]:
import pandas as pd
import numpy as np

# Sample DataFrame
data = {
    'Column1': [1234567.89, -9876543.21, 3456789.01],
    'Column2': [-12345.67, 98765.43, -56789.01]
}
df = pd.DataFrame(data)

# Step 1: Set display options for large numbers with commas
pd.set_option('display.float_format', '{:,.2f}'.format)

# Step 2: Define a styling function for red negative numbers with parentheses
def format_negative_red(value):
    if value < 0:
        return f'color: red; font-weight: bold;'  # CSS for red color
    return None

def format_negative_parentheses(value):
    return f'({abs(value):,.2f})' if value < 0 else f'{value:,.2f}'

def display_df(df):
    return df.style.applymap(format_negative_red).format(format_negative_parentheses)

# Apply formatting for display
# styled_df = df.style.applymap(format_negative_red).format(format_negative_parentheses)

# Display the styled DataFrame in Jupyter Notebook or save as HTML
# styled_df.to_html("styled_output.html")  # Save as HTML for visualization

display_df(df)


/var/folders/_2/m6nqqq692z5gjqwq_qk2lcjr0000gn/T/ipykernel_1962/1932810938.py:24: FutureWarning:

Styler.applymap has been deprecated. Use Styler.map instead.



,Column1,Column2
0,"1,234,567.89","(12,345.67)"
1,"(9,876,543.21)","98,765.43"
2,"3,456,789.01","(56,789.01)"


In [36]:
# df_melted.head(50)
display(df)

df_melted

,Year,United States,China,Japan,Germany,India,United Kingdom,France,Brazil,Canada,Italy
0,2000,10.0,1.2,4.5,2.5,0.5,2.30,2.10,1.8,1.60,1.90
1,2001,11.2,2.7,4.6,2.7,0.8,2.45,2.22,1.9,1.68,1.95
2,2002,12.4,4.2,4.7,2.9,1.1,2.60,2.34,2.0,1.76,2.00
3,2003,13.6,5.7,4.8,3.1,1.4,2.75,2.46,2.1,1.84,2.05
4,2004,14.8,7.2,4.9,3.3,1.7,2.90,2.58,2.2,1.92,2.10
5,2005,16.0,8.7,5.0,3.5,2.0,3.05,2.70,2.3,2.00,2.15
6,2006,17.2,10.2,5.1,3.7,2.3,3.20,2.82,2.4,2.08,2.20
7,2007,18.4,11.7,5.2,3.9,2.6,3.35,2.94,2.5,2.16,2.25
8,2008,19.6,13.2,5.3,4.1,2.9,3.50,3.06,2.6,2.24,2.30
9,2009,20.8,14.7,5.4,4.3,3.2,3.65,3.18,2.7,2.32,2.35


,Year,Country,GDP
0,2000,United States,10.00
1,2001,United States,11.20
2,2002,United States,12.40
3,2003,United States,13.60
4,2004,United States,14.80
...,...,...,...
245,2020,Italy,2.90
246,2021,Italy,2.95
247,2022,Italy,3.00
248,2023,Italy,3.05


In [ ]:
file_path = os.path.join(output_folder, "sp500_constituents.csv")

In [ ]:
folder_name = "data"
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

In [28]:
output_folder = os.path.join("data", "gdp_data")
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
output_folder

'data/gdp_data'

/var/folders/_2/m6nqqq692z5gjqwq_qk2lcjr0000gn/T/ipykernel_1962/2910871687.py:33: FutureWarning:

Styler.applymap has been deprecated. Use Styler.map instead.



,Column1,Column2
0,"1,234,568","(-12,346)"
1,"(-9,876,543)","98,765"
2,"3,456,789","(-56,789)"
3,"100,000","123,457"
